# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
airbnb = pd.read_csv('airbnb_hw.csv')
airbnb.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [21]:
# Display unique values in the 'Price' column before cleaning.
print("Unique values in 'Price' before cleaning:", airbnb['Price'].unique(),'\n')

# Clean the 'Price' variable by removing '$' and ',' and converting to numeric, coercing errors to NaN.
airbnb['Price'] = airbnb['Price'].astype(str).str.replace('[$,]', '', regex=True)
airbnb['Price'] = pd.to_numeric(airbnb['Price'], errors='coerce')

# Calculate the number of missing values after cleaning.
missing_prices_count = airbnb['Price'].isnull().sum()

print(f"Number of missing values in the cleaned 'Price' variable: {missing_prices_count}")

Unique values in 'Price' before cleaning: [  145    37    28   199   549   149   250    90   270   290   170    59
    49    68   285    75   100   150   700   125   175    40    89    95
    99   499   120    79   110   180   143   230   350   135    85    60
    70    55    44   200   165   115    74    84   129    50   185    80
   190   140    45    65   225   600   109  1990    73   240    72   105
   155   160    42   132   117   295   280   159   107    69   239   220
   399   130   375   585   275   139   260    35   133   300   289   179
    98   195    29    27    39   249   192   142   169  1000   131   138
   113   122   329   101   475   238   272   308   126   235   315   248
   128    56   207   450   215   210   385   445   136   247   118    77
    76    92   198   205   299   222   245   104   153   349   114   320
   292   226   420   500   325   307    78   265   108   123   189    32
    58    86   219   800   335    63   229   425    67    87  1200   158
   650   

In [26]:
police = pd.read_csv('mn_police_use_of_force.csv')
police.head()

,response_datetime,problem,is_911_call,primary_offense,subject_injury,force_type,force_type_action,race,sex,age,type_resistance,precinct,neighborhood
0,2016/01/01 00:47:36,Assault in Progress,Yes,DASLT1,NaN,Bodily Force,Body Weight to Pin,Black,Male,20.0,Tensed,1,Downtown East
1,2016/01/01 02:19:34,Fight,No,DISCON,NaN,Chemical Irritant,Personal Mace,Black,Female,27.0,Verbal Non-Compliance,1,Downtown West
2,2016/01/01 02:19:34,Fight,No,DISCON,NaN,Chemical Irritant,Personal Mace,White,Female,23.0,Verbal Non-Compliance,1,Downtown West
3,2016/01/01 02:28:48,Fight,No,PRIORI,NaN,Chemical Irritant,Crowd Control Mace,Black,Male,20.0,Commission of Crime,1,Downtown West
4,2016/01/01 02:28:48,Fight,No,PRIORI,NaN,Chemical Irritant,Crowd Control Mace,Black,Male,20.0,Commission of Crime,1,Downtown West


In [23]:
# Display unique values and value counts in the original 'subject_injury' column.
print("Unique values in original 'subject_injury':", police['subject_injury'].unique(),'\n')
print("Value counts in original 'subject_injury' (including NaN):\n", police['subject_injury'].value_counts(dropna=False), '\n')

# Clean the 'subject_injury' variable: replace missing values with 'Unknown'.
police['subject_injury_cleaned'] = police['subject_injury'].apply(lambda x: 'Yes' if x != 'No' and pd.notna(x) else ('No' if x == 'No' else 'Unknown'))

# Display unique values and value counts in the cleaned 'subject_injury' column
print("Unique values in cleaned 'subject_injury':", police['subject_injury_cleaned'].unique(),'\n')
print("Value counts in cleaned 'subject_injury':\n", police['subject_injury_cleaned'].value_counts(), '\n')


# Calculate the proportion of missing values in the original 'subject_injury' column
missing_proportion = police['subject_injury'].isnull().sum() / len(police)
print(f"Proportion of missing values in 'subject_injury': {missing_proportion:.2f}\n")

# Cross-tabulate the cleaned 'subject_injury' variable with 'force_type'
cross_tab = pd.crosstab(police['subject_injury_cleaned'], police['force_type'], margins=True, margins_name='Total')
print("Cross-tabulation of cleaned 'subject_injury' and 'force_type':\n", cross_tab)

# Based on the cross-tabulation, you can observe if certain 'force_type' values have a higher proportion of 'Unknown' in 'subject_injury'.
# A high proportion of 'Unknown' for a specific 'force_type' could indicate patterns in when the data are missing.

Proportion of missing values in 'subject_injury': 0.76

Cross-tabulation of cleaned 'subject_injury' and 'force_type':
 force_type              Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury_cleaned                                                    
No                          0          1093                131        2   
Unknown                     2          7051               1421        0   
Yes                         2          1286                 41        0   
Total                       4          9430               1593        2   

force_type              Gun Point Display  Improvised Weapon  Less Lethal  \
subject_injury_cleaned                                                      
No                                     33                 34            0   
Unknown                                27                 74           87   
Yes                                    44                 40            0   
Total                                 104   

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)